# Keras implementation of ResNet by Jingchen Feng

   ## Highlight:
1. Image classification with ResNet on Cifar10 dataset with implementation in Keras
2. Reproduce the result in the paper https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf
3. Implement the ResNet with bottleNeck layers. 
4. Apply L2 regularization and data augmentation to fight overfitting
5. build an ensemble model of a 20 layer ResNet, a 32 layer ResNet and a 29 layer ResNet with bottleNeck layers to reach a testing accuracy of 93.5%

In [ ]:
# A summary of the results
# Model type          #layers             #accuracy            #parameters        #L2 weight

#  ResNet               20                  91.3%                 0.27M              3e-4
#  ResNet               32                  92.1%                 0.46M              1e-4
#  ResNet with                                    
#  bottleneck layers    29                  91.7%                 0.49M              1e-4
#  ensemble             NA                  93.5%                 

In [ ]:
# This script is to show the result of trained indivisual models and the ensembled model
# for model construction details, see Construct_model.ipynb
import read_file
from keras.models import Model, load_model
import numpy as np

# download cifar 10 at: https://www.cs.toronto.edu/~kriz/cifar.html
img_path = './cifar-10-batches-py/'    #the file path of the training & testing data

In [ ]:
# read the data with img_read function defined in read_file.py
(x_train, y_train), (x_test, y_test) = read_file.img_read(img_path)    

In [ ]:
model_l20= load_model('cifar10_resNet20.h5')

model_l32= load_model('cifar10_resNet32.h5')   # read the pretrained model (32 layer ResNet)

model_l20_bottleneck= load_model('cifar10_resNet20_bottleneck.h5')

In [ ]:
model_l32.summary()       # see the network architecture

In [ ]:
scores_l32 = model_l32.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_l32[0])
print('Test accuracy:', scores_l32[1])

In [ ]:
# this function is used to calculate the accuracy of the prediction. Note the dimension of y is (10000, 10)
def categorical_accuracy(y_true, y_pred):
    y_true_class= y_true.argmax(axis=-1)
    y_pred_class= y_pred.argmax(axis=-1)
    match= np.sum(y_true_class== y_pred_class)
    return match/y_true.shape[0]

In [ ]:
# I did not tune the weights, we can actually reach better testing accuracy by tuning weights with training data
weights= [0.3333, 0.3333, 0.3333];  

models=[model_l20, model_l20_bottleneck, model_l32]
results= np.zeros(y_test.shape)

for (idx, model) in enumerate(models):
    pred= model.predict(x_test, verbose=1)
    
    results+= weights[idx]*pred
    
print('Test accuracy of ensembled model= '+ str(categorical_accuracy(y_test, results)))